In [ ]:
import os
import pandas as pd
from pathlib import Path
import openai

import sys
sys.path.insert(0, '..')
from utils_sepaq import PARCS

In [ ]:
PATH_DATA = Path("../data/randos")
MODEL_NAME = "text-embedding-ada-002"

In [ ]:
list_files = os.listdir(PATH_DATA)

In [ ]:
def format_treck_text(row: pd.core.series.Series):
    text = ""
    for col in row.index:
        text = text + f"{col}: {row[col]}\n"
    return text
    

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
embeddings_list = []
parc_list = []
text_list = []

for file in list_files:
    parc_name = file.replace(".csv", "")
    df_parc = pd.read_csv(Path(PATH_DATA, file))
    
    for _, row in df_parc.iterrows():
        text = format_treck_text(row)
        text_list.append(text)
        embeddings_list.append(get_embedding(text))
        parc_list.append(parc_name)

In [ ]:
df_embeddings = pd.DataFrame(embeddings_list).add_prefix("emb_")
df_embeddings["parc"] = parc_list
df_embeddings["text"] = text_list
df_embeddings

In [ ]:
df_embeddings.parc.value_counts()

In [ ]:
df_embeddings.to_csv("../df_embeddings.csv")

In [ ]:
prompt = "J'aimerais une randonnée facile ou les chiens sont acceptés et ou nous pourrons amener notre poussette."
prompt_embedding = get_embedding(prompt)

In [ ]:
emb_cols = [col for col in df_embeddings if col.startswith('emb_')]

In [ ]:
from openai.embeddings_utils import cosine_similarity
test_emb = df_embeddings[emb_cols].apply(lambda x: cosine_similarity(x, prompt_embedding), axis=1)

In [ ]:
def search_docs(doc_df, q_embedding, max_rando=20):
    emb_cols = [col for col in doc_df if col.startswith('emb_')]
    test_emb = df_embeddings[emb_cols].apply(lambda x: cosine_similarity(x, prompt_embedding), axis=1)
    res = df_embeddings.iloc[test_emb.sort_values(ascending=False).index][0:(max_rando-1)][["parc", "text"]]
    return res

test = search_docs(df_embeddings, prompt_embedding)

In [ ]:
for _, row in test.iterrows():
    doc = ""
    print(row)
    

In [ ]:
df_embeddings.iloc[test_emb.sort_values(ascending=False).index].text[6]